In [1]:
!pip install trl

In [2]:
!pip install -U bitsandbytes

In [3]:
!pip install torch

In [4]:
!pip install peft

In [5]:
from trl import SFTTrainer

2025-09-22 13:19:35.997381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758547176.009966    3356 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758547176.013790    3356 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-22 13:19:36.026589: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import pandas as pd

In [7]:
from datasets import load_dataset
from datasets import Dataset
import json

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./local_model_dir", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "./local_model_dir",
    trust_remote_code=True,
)

# Get the device from the model (for quantized models, this is usually 'cuda' if available)
#device = next(model.parameters()).device

In [9]:
train_df = pd.read_csv("./data/train_dataset.csv")

In [10]:
train_df = Dataset.from_pandas(train_df)

In [11]:
SYSTEM_PROMPT = "You are a helpful assistant that uses the provided tools to answer."

In [12]:
def format_dataset(example):
    query = example['query']
    tools = json.loads(example['tools'])
    expected_calls = json.loads(example['answers'])
    #print([call for call in expected_calls])
    formatted_calls = "".join([f"<tool_call>{json.dumps(call)}</tool_call>" for call in expected_calls])
    #print(formatted_calls)
                               
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query}, 
        {"role": "assistant", "content": formatted_calls}
    ]
    
    texts = tokenizer.apply_chat_template(messages,tools = tools, tokenize=False, add_generation_prompt=False)
    return {"text": texts}

In [13]:
processed_train = train_df.map(format_dataset, remove_columns=train_df.column_names) 

Map:   0%|          | 0/489 [00:00<?, ? examples/s]

In [14]:
processed_train[5]

{'text': '<|im_start|>system\nYou are a helpful assistant that uses the provided tools to answer.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"name": "chi_square_independence_test", "description": "Performs a Chi-Square test for independence on a 2x2 contingency table.", "parameters": {"contingency_table": {"description": "A 2x2 contingency table represented as a list of lists.", "type": "List[List[int]]", "default": 0.05}, "significance_level": {"description": "The significance level for the Chi-Square test. Defaults to 0.05.", "type": "float, optional", "default": 0.05}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\nA psychologist wants to test if there is a sig

In [15]:
from peft import LoraConfig
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "k_proj",
        "up_proj",
        "down_proj",
        "gate_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [16]:
model.add_adapter(lora_config, adapter_name="lora_1")

In [17]:
model.gradient_checkpointing_enable()


In [18]:
from trl import SFTConfig

In [19]:
training_args = SFTConfig(packing=False)

In [20]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_train,
)

Adding EOS to train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,2.669100
20,1.567100
30,0.896000
40,0.571500
50,0.457800
60,0.391600
70,0.346000
80,0.296700
90,0.276500
100,0.243300


TrainOutput(global_step=186, training_loss=0.5078044251729084, metrics={'train_runtime': 955.7064, 'train_samples_per_second': 1.535, 'train_steps_per_second': 0.195, 'total_flos': 1.1597077511384064e+16, 'train_loss': 0.5078044251729084, 'entropy': 0.20664420227209726, 'num_tokens': 397656.0, 'mean_token_accuracy': 0.9527674118677775, 'epoch': 3.0})

In [ ]:
torch.cuda.memory._record_memory_history()
torch.cuda.memory._dump_snapshot("snapshot.pickle")

In [ ]:
print(torch.cuda.memory._record_memory_history())

None


In [26]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1): Identity()
            )
            (lora_A): ModuleDict(
              (lora_1): Linear(in_features=2560, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (lora_1): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1):

In [27]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1): Identity()
            )
            (lora_A): ModuleDict(
              (lora_1): Linear(in_features=2560, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (lora_1): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (lora_1):

In [28]:
trainer.save_model('./finetuned_qwen4b')